In [1]:
import pandas
from RexR import *
import _helpers
Rocket = RexR(datalocation = '/home/bramiozo/DEV/RexR/_data/genomic_data/data.pkl', seed = 3123, debug = True)

++++++++++++++++++++++++++++++ Firing up RexR! ++++++++++++++++++++++++++++++


In [ ]:
##### LOAD IN DATA
###################

In [2]:
Rocket.load_probeset_data();

In [ ]:
class_model = Rocket.classify_treatment(model='ensemble', 
                                              features = 'all', 
                                              grouping= 'first', 
                                              n_splits = 10, 
                                              reduction = None)

++++++++++++++++++++++++++++++  RESULTS FOR CLASSIFICATION WITH GENOMIC DATA ++++++++++++++++++++++++++++++
MODEL: SVM 0.689655172414
++++++++++++++++++++++++++++++  Report ++++++++++++++++++++++++++++++
             precision    recall  f1-score   support

          0       0.76      0.67      0.71        33
          1       0.62      0.72      0.67        25

avg / total       0.70      0.69      0.69        58

MODEL: LogisticRegression 0.689655172414
++++++++++++++++++++++++++++++  Report ++++++++++++++++++++++++++++++
             precision    recall  f1-score   support

          0       0.76      0.67      0.71        33
          1       0.62      0.72      0.67        25

avg / total       0.70      0.69      0.69        58

MODEL: RandomForest 0.637931034483
++++++++++++++++++++++++++++++  Report ++++++++++++++++++++++++++++++
             precision    recall  f1-score   support

          0       0.63      0.88      0.73        33
          1       0.67      0.32      0.43 

In [ ]:
class_model = Rocket.classify_treatment(model='ensemble', 
                                              features = 'all', 
                                              grouping= 'mean', 
                                              n_splits = 10, 
                                              reduction = None)

++++++++++++++++++++++++++++++  RESULTS FOR CLASSIFICATION WITH GENOMIC DATA ++++++++++++++++++++++++++++++
MODEL: SVM 0.68115942029
++++++++++++++++++++++++++++++  Report ++++++++++++++++++++++++++++++
             precision    recall  f1-score   support

          0       0.82      0.57      0.68        40
          1       0.59      0.83      0.69        29

avg / total       0.72      0.68      0.68        69

MODEL: LogisticRegression 0.666666666667
++++++++++++++++++++++++++++++  Report ++++++++++++++++++++++++++++++
             precision    recall  f1-score   support

          0       0.79      0.57      0.67        40
          1       0.57      0.79      0.67        29

avg / total       0.70      0.67      0.67        69

MODEL: RandomForest 0.724637681159
++++++++++++++++++++++++++++++  Report ++++++++++++++++++++++++++++++
             precision    recall  f1-score   support

          0       0.71      0.90      0.79        40
          1       0.78      0.48      0.60  

In [ ]:
class_model = Rocket.classify_treatment(model='ensemble', 
                                              features = 'all', 
                                              grouping= 'min', 
                                              n_splits = 10, 
                                              reduction = None)